In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description

In [2]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Return weather for the input city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to return its weather"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [9]:
developer_prompt = """
You're a weather forecasting bot. 
You're given a city name and your task is to return the weather in that city.

Use the tools that are provided for you to return the weather for that city.
""".strip()

In [10]:
from openai import OpenAI
client = OpenAI()

In [12]:
import chat_assistant

tools =chat_assistant.Tools()

tools.add_tool(get_weather, get_weather_tool)

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt= developer_prompt,
    chat_interface= chat_interface,
    client= client
    
)

In [13]:
chat.run()

Chat ended.


### Q2. Adding another tool

In [17]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set the temperature as the weather for the input city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to set its weather"
            },
            "temp": {
                "type": "number",
                "description": "The temperature to set as the city's weather"
            }
        },
        "required": ["city","temp"],
        "additionalProperties": False
    }
}


tools.add_tool(set_weather, set_weather_tool)

In [20]:
chat.run()

Chat ended.


In [21]:
known_weather_data

{'berlin': 20.0, 'paris': 34.1}

### Q3. Install FastMCP

In [22]:
!pip install fastmcp

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   --------------------------- ------------ 2.4/3.4 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 3.4/3.4 MB 11.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/587.4 kB ? eta -:--:--
   ---------------------------------------- 587.4/587.4 kB 7.2 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11117 sha256=b703d851d0877bbf21830a39af90d0ed309e01634ab1d2db6f4306ea96d4dbc0
  Stored in directory: c:\users\eghiasva\appdata\local\package

In [24]:
import fastmcp
print(fastmcp.__version__)

2.10.5


### Q4. Simple MCP Server

In [27]:
!python ./weather_server.py

^C


### Q5. Protocol

In [28]:
""" 
asking the weather in berlin from the MCP server: 
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city":"berlin"}}}

The answer:
{"jsonrpc":"2.0","id":4,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}
"""

' \nasking the weather in berlin from the MCP server: \n{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city":"berlin"}}}\n\nThe answer:\n{"jsonrpc":"2.0","id":4,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}\n'

### Q6. Client

In [32]:
from fastmcp import Client
import weather_server


async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        for tool in tools:
            print(tool)

await main()

name='get_weather' title=None description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.' inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True} annotations=None meta=None
name='set_weather' title=None description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update." inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}

### Using tools from the MCP server (optional)

In [33]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo ðŸš€', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully


In [34]:
our_mcp_client.get_tools()
our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})

Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


{'content': [{'type': 'text', 'text': '20.0'}],
 'structuredContent': {'result': 20.0},
 'isError': False}

In [35]:
import json

class MCPTools:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        self.tools = None
    
    def get_tools(self):
        if self.tools is None:
            mcp_tools = self.mcp_client.get_tools()
            self.tools = convert_tools_list(mcp_tools)
        return self.tools

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        result = self.mcp_client.call_tool(function_name, arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }

In [36]:
our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo ðŸš€', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


Chat ended.
